# Project: Identify Customer Segments

In this project, you will apply unsupervised learning techniques to identify segments of the population that form the core customer base for a mail-order sales company in Germany. These segments can then be used to direct marketing campaigns towards audiences that will have the highest expected rate of returns. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

This notebook will help you complete this task by providing a framework within which you will perform your analysis steps. In each step of the project, you will see some text describing the subtask that you will perform, followed by one or more code cells for you to complete your work. **Feel free to add additional code and markdown cells as you go along so that you can explore everything in precise chunks.** The code cells provided in the base template will outline only the major tasks, and will usually not be enough to cover all of the minor tasks that comprise it.

It should be noted that while there will be precise guidelines on how you should handle certain tasks in the project, there will also be places where an exact specification is not provided. **There will be times in the project where you will need to make and justify your own decisions on how to treat the data.** These are places where there may not be only one way to handle the data. In real-life tasks, there may be many valid ways to approach an analysis task. One of the most important things you can do is clearly document your approach so that other scientists can understand the decisions you've made.

At the end of most sections, there will be a Markdown cell labeled **Discussion**. In these cells, you will report your findings for the completed section, as well as document the decisions that you made in your approach to each subtask. **Your project will be evaluated not just on the code used to complete the tasks outlined, but also your communication about your observations and conclusions at each stage.**

In [95]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

'''
Import note: The classroom currently uses sklearn version 0.19.
If you need to use an imputer, it is available in sklearn.preprocessing.Imputer,
instead of sklearn.impute as in newer versions of sklearn.
'''

'\nImport note: The classroom currently uses sklearn version 0.19.\nIf you need to use an imputer, it is available in sklearn.preprocessing.Imputer,\ninstead of sklearn.impute as in newer versions of sklearn.\n'

### Step 0: Load the Data

There are four files associated with this project (not including this one):

- `Udacity_AZDIAS_Subset.csv`: Demographics data for the general population of Germany; 891211 persons (rows) x 85 features (columns).
- `Udacity_CUSTOMERS_Subset.csv`: Demographics data for customers of a mail-order company; 191652 persons (rows) x 85 features (columns).
- `Data_Dictionary.md`: Detailed information file about the features in the provided datasets.
- `AZDIAS_Feature_Summary.csv`: Summary of feature attributes for demographics data; 85 features (rows) x 4 columns

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. You will use this information to cluster the general population into groups with similar demographic properties. Then, you will see how the people in the customers dataset fit into those created clusters. The hope here is that certain clusters are over-represented in the customers data, as compared to the general population; those over-represented clusters will be assumed to be part of the core userbase. This information can then be used for further applications, such as targeting for a marketing campaign.

To start off with, load in the demographics data for the general population into a pandas DataFrame, and do the same for the feature attributes summary. Note for all of the `.csv` data files in this project: they're semicolon (`;`) delimited, so you'll need an additional argument in your [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call to read in the data properly. Also, considering the size of the main dataset, it may take some time for it to load completely.

Once the dataset is loaded, it's recommended that you take a little bit of time just browsing the general structure of the dataset and feature summary file. You'll be getting deep into the innards of the cleaning in the first major step of the project, so gaining some general familiarity can help you get your bearings.

In [96]:
# Load in the general demographics data.
azdias = pd.read_csv('Udacity_AZDIAS_Subset.csv', sep=';')
customer = pd.read_csv('Udacity_CUSTOMERS_Subset.csv', sep= ';')
# Load in the feature summary file.
feat_info = pd.read_csv('AZDIAS_Feature_Summary.csv', sep =';')

In [97]:
print('Number of columns:', len(azdias.columns))
print('Number of rows:', len(azdias))
azdias.info()



Number of columns: 85
Number of rows: 891221
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Data columns (total 85 columns):
AGER_TYP                 891221 non-null int64
ALTERSKATEGORIE_GROB     891221 non-null int64
ANREDE_KZ                891221 non-null int64
CJT_GESAMTTYP            886367 non-null float64
FINANZ_MINIMALIST        891221 non-null int64
FINANZ_SPARER            891221 non-null int64
FINANZ_VORSORGER         891221 non-null int64
FINANZ_ANLEGER           891221 non-null int64
FINANZ_UNAUFFAELLIGER    891221 non-null int64
FINANZ_HAUSBAUER         891221 non-null int64
FINANZTYP                891221 non-null int64
GEBURTSJAHR              891221 non-null int64
GFK_URLAUBERTYP          886367 non-null float64
GREEN_AVANTGARDE         891221 non-null int64
HEALTH_TYP               891221 non-null int64
LP_LEBENSPHASE_FEIN      886367 non-null float64
LP_LEBENSPHASE_GROB      886367 non-null float64
LP_FAMILIE_FEIN          886367 non-n

In [98]:
print('Number of columns:', len(feat_info.columns))
print('Number of rows:', len(feat_info))
feat_info.info()

Number of columns: 4
Number of rows: 85
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 4 columns):
attribute             85 non-null object
information_level     85 non-null object
type                  85 non-null object
missing_or_unknown    85 non-null object
dtypes: object(4)
memory usage: 2.7+ KB


In [99]:
azdias.isna()


,AGER_TYP,ALTERSKATEGORIE_GROB,ANREDE_KZ,CJT_GESAMTTYP,FINANZ_MINIMALIST,FINANZ_SPARER,FINANZ_VORSORGER,FINANZ_ANLEGER,FINANZ_UNAUFFAELLIGER,FINANZ_HAUSBAUER,...,PLZ8_ANTG1,PLZ8_ANTG2,PLZ8_ANTG3,PLZ8_ANTG4,PLZ8_BAUMAX,PLZ8_HHZ,PLZ8_GBZ,ARBEIT,ORTSGR_KLS9,RELAT_AB
0,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [100]:
feat_info.isnull()

,attribute,information_level,type,missing_or_unknown
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
5,False,False,False,False
6,False,False,False,False
7,False,False,False,False
8,False,False,False,False
9,False,False,False,False


In [101]:
answer = feat_info.isnull().sum().sum()
print("number of nulls in original dataset:", answer)

number of nulls in original dataset: 0


In [102]:
info = azdias.isnull().sum().sum()
print("number of nulls in original dataset:", info)

number of nulls in original dataset: 4896838


> **Tip**: Add additional cells to keep everything in reasonably-sized chunks! Keyboard shortcut `esc --> a` (press escape to enter command mode, then press the 'A' key) adds a new cell before the active cell, and `esc --> b` adds a new cell after the active cell. If you need to convert an active cell to a markdown cell, use `esc --> m` and to convert to a code cell, use `esc --> y`. 

## Step 1: Preprocessing

### Step 1.1: Assess Missing Data

The feature summary file contains a summary of properties for each demographics data column. You will use this file to help you make cleaning decisions during this stage of the project. First of all, you should assess the demographics data in terms of missing data. Pay attention to the following points as you perform your analysis, and take notes on what you observe. Make sure that you fill in the **Discussion** cell with your findings and decisions at the end of each step that has one!

#### Step 1.1.1: Convert Missing Value Codes to NaNs
The fourth column of the feature attributes summary (loaded in above as `feat_info`) documents the codes from the data dictionary that indicate missing or unknown data. While the file encodes this as a list (e.g. `[-1,0]`), this will get read in as a string object. You'll need to do a little bit of parsing to make use of it to identify and clean the data. Convert data that matches a 'missing' or 'unknown' value code into a numpy NaN value. You might want to see how much data takes on a 'missing' or 'unknown' code, and how much data is naturally missing, as a point of interest.

**As one more reminder, you are encouraged to add additional cells to break up your analysis into manageable chunks.**

In [ ]:
# Identify missing or unknown data values and convert them to NaNs.

for x in range(len(feat_info)):
    null_data = feat_info.iloc[x]['missing_or_unknown']
    null_data = null_data.strip('[')
    null_data = null_data.strip(']')
    null_data = null_data.split(sep=',')
    null_data = [int(value) if (value!='X' and value!='XX' and value!='') else value for value in null_data]
    if null_data != ['']:
        azdias = azdias.replace({feat_info.iloc[x]['attribute']: null_data}, np.nan)


In [ ]:
answer1 = azdias.isnull().sum().sum()
print("number of nulls in changed dataset:", answer1)

#### Step 1.1.2: Assess Missing Data in Each Column

How much missing data is present in each column? There are a few columns that are outliers in terms of the proportion of values that are missing. You will want to use matplotlib's [`hist()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function to visualize the distribution of missing value counts to find these columns. Identify and document these columns. While some of these columns might have justifications for keeping or re-encoding the data, for this project you should just remove them from the dataframe. (Feel free to make remarks about these outlier columns in the discussion, however!)

For the remaining features, are there any patterns in which columns have, or share, missing data?

In [ ]:
null_percent = (azdias.isnull().sum() * 100/len(azdias))
print("Percentage of missing values:", null_percent)

In [ ]:
max_null = null_percent.max()
print("The highest percent of nulls in a column is:", max_null)

In [ ]:
# Perform an assessment of how much missing data there is in each column of the
# dataset.
plt.hist(null_percent, bins = 50)


In [ ]:
# Investigate patterns in the amount of missing data in each column.
azdias.isnull().sum()[azdias.isnull().sum()>0].plot(kind='barh', figsize=(10, 40))
plt.xticks(rotation='horizontal')


In [ ]:
# Remove the outlier columns from the dataset. (You'll perform other data
# engineering tasks such as re-encoding and imputation later.)

#removing these columns with high null values 
new_azdias = azdias.drop(labels = ['AGER_TYP', 'GEBURTSJAHR', 'TITEL_KZ', 'KK_KUNDENTYP', 'KBA05_BAUMAX', 'ALTER_HH'], axis =1 )


In [ ]:
print('Number of columns:', len(new_azdias.columns))



#### Discussion 1.1.2: Assess Missing Data in Each Column

After investigating the null values of the data i removed the following columns: 'AGER_TYP', 'GEBURTSJAHR', 'TITEL_KZ', 'KK_KUNDENTYP', 'KBA05_BAUMAX', 'ALTER_HH', as they had the highest amount of null values. With that many nulls I didn't feel that they added any value to the data set. I didn't notice any patterns in the data, but it seems as though there are only a few columns without any null data, and a lot of coulmns that have nulls. 

#### Step 1.1.3: Assess Missing Data in Each Row

Now, you'll perform a similar assessment for the rows of the dataset. How much data is missing in each row? As with the columns, you should see some groups of points that have a very different numbers of missing values. Divide the data into two subsets: one for data points that are above some threshold for missing values, and a second subset for points below that threshold.

In order to know what to do with the outlier rows, we should see if the distribution of data values on columns that are not missing data (or are missing very little data) are similar or different between the two groups. Select at least five of these columns and compare the distribution of values.
- You can use seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) function to create a bar chart of code frequencies and matplotlib's [`subplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html) function to put bar charts for the two subplots side by side.
- To reduce repeated code, you might want to write a function that can perform this comparison, taking as one of its arguments a column to be compared.

Depending on what you observe in your comparison, this will have implications on how you approach your conclusions later in the analysis. If the distributions of non-missing features look similar between the data with many missing values and the data with few or no missing values, then we could argue that simply dropping those points from the analysis won't present a major issue. On the other hand, if the data with many missing values looks very different from the data with few or no missing values, then we should make a note on those data as special. We'll revisit these data later on. **Either way, you should continue your analysis for now using just the subset of the data with few or no missing values.**

In [ ]:
# How much data is missing in each row of the dataset?
find_row_null=new_azdias.isnull().sum(axis =1)


In [ ]:
plt.hist(find_row_null, bins = 10)

In [ ]:
max_null = new_azdias.isnull().sum(axis = 1).max()
min_null = new_azdias.isnull().sum(axis = 1).min()
print("max null :" ,max_null)
print('min null:', min_null)

In [ ]:
# Write code to divide the data into two subsets based on the number of missing
# values in each row.
#Based on the plt above I will divide into two set min = less than 30, and max =grater or equal 30

min_null_split =  new_azdias[find_row_null<30].reset_index()
min_null_split.isnull().sum()



In [ ]:
max_null_split =  new_azdias[find_row_null>=30].reset_index()

In [ ]:
# Compare the distribution of values for at least five columns where there are
# no or few missing values, between the two subsets.
column_list = ['SEMIO_SOZ'  # created a list of columns from min_null_split to have columns with low number of nulls.             
,'SEMIO_FAM'                    
,'SEMIO_REL'                    
,'SEMIO_MAT'                    
,'SEMIO_VERT'                   
,'SEMIO_LUST'                   
,'SEMIO_ERL', 'GEBAEUDETYP_RASTER','INNENSTADT']
plt.figure(figsize =(15,20))
for i in enumerate(column_list):
    plt.subplot(6,3,i[0] +1)
    sns.countplot(i[1], data =new_azdias )

#### Discussion 1.1.3: Assess Missing Data in Each Row

From the above plots we can see that the distribution of these columns are fairly equally distributed, the only exception is Gebaeudetyp_raster this distribution has the lowest value and highest value count within the same plot.  

### Step 1.2: Select and Re-Encode Features

Checking for missing data isn't the only way in which you can prepare a dataset for analysis. Since the unsupervised learning techniques to be used will only work on data that is encoded numerically, you need to make a few encoding changes or additional assumptions to be able to make progress. In addition, while almost all of the values in the dataset are encoded using numbers, not all of them represent numeric values. Check the third column of the feature summary (`feat_info`) for a summary of types of measurement.
- For numeric and interval data, these features can be kept without changes.
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling may be necessary for the remaining two variable types: categorical, and 'mixed'.

In the first two parts of this sub-step, you will perform an investigation of the categorical and mixed-type features and make a decision on each of them, whether you will keep, drop, or re-encode each. Then, in the last part, you will create a new data frame with only the selected and engineered columns.

Data wrangling is often the trickiest part of the data analysis process, and there's a lot of it to be done here. But stick with it: once you're done with this step, you'll be ready to get to the machine learning parts of the project!

In [ ]:
# How many features are there of each data type?
feat_info['type'].value_counts()


#### Step 1.2.1: Re-Encode Categorical Features

For categorical data, you would ordinarily need to encode the levels as dummy variables. Depending on the number of categories, perform one of the following:
- For binary (two-level) categoricals that take numeric values, you can keep them without needing to do anything.
- There is one binary variable that takes on non-numeric values. For this one, you need to re-encode the values as numbers or create a dummy variable.
- For multi-level categoricals (three or more values), you can choose to encode the values using multiple dummy variables (e.g. via [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)), or (to keep things straightforward) just drop them from the analysis. As always, document your choices in the Discussion section.

In [ ]:
# Assess categorical variables: which are binary, which are multi-level, and
# which one needs to be re-encoded?

feat_info[feat_info['type'] == 'categorical']

In [ ]:
feat_info[feat_info['type'] == 'mixed']

In [ ]:
#dropping categorical and attribute columns as they are not needed.
col_to_drop =feat_info[feat_info['type'] == 'categorical']['attribute']
for x in col_to_drop:
    if x in min_null_split:
        min_null_split.drop(x, inplace = True, axis = 1)
  
       

In [ ]:

#min_null_split.head()
#set(min_null_split)

#### Discussion 1.2.1: Re-Encode Categorical Features

For this section I decided to drop the catogorical and attribute types, to keep the data simple and striaght forward. We will be keeping the mixed portion so that the vairables "PRAEGENDE_JUGENDJAHRE" and "CAMEO_INTL_2015" can be re-coded and re-enginered below. 

#### Step 1.2.2: Engineer Mixed-Type Features

There are a handful of features that are marked as "mixed" in the feature summary that require special treatment in order to be included in the analysis. There are two in particular that deserve attention; the handling of the rest are up to your own choices:
- "PRAEGENDE_JUGENDJAHRE" combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, you should create two new variables to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).
- If you decide to keep or engineer new features around the other mixed-type features, make sure you note your steps in the Discussion section.

Be sure to check `Data_Dictionary.md` for the details needed to finish these tasks.

In [ ]:
# Re-encode categorical variable(s) to be kept in the analysis.

min_null_split.PRAEGENDE_JUGENDJAHRE.value_counts()

In [ ]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.
decadesKey = {
    1: 40,
    2: 40,
    3: 50,
    4: 50,
    5: 60,
    6: 60,
    7: 60,
    8: 70,
    9: 70,
    10: 80,
    11: 80,
    12: 80,
    13: 80,
    14: 90,
    15: 90
}

decades = [None if pd.isna(p) else decadesKey[p] for p in list(min_null_split.PRAEGENDE_JUGENDJAHRE)]
min_null_split['DECADE'] = decades


In [ ]:
movementsKey = {
    1: 'M',
    2: 'A',
    3: 'M',
    4: 'A',
    5: 'M',
    6: 'A',
    7: 'A',
    8: 'M',
    9: 'A',
    10: 'M',
    11: 'A',
    12: 'M',
    13: 'A',
    14: 'M',
    15: 'A'
}

movements = [None if pd.isna(p) else movementsKey[p] for p in list(min_null_split.PRAEGENDE_JUGENDJAHRE)]
min_null_split['MOVEMENT'] = movements

In [ ]:
# Investigate "CAMEO_INTL_2015" and engineer two new variables.

tensKey = {
    1: None,
    '11': 1,
    '12': 1,
    '13': 1,
    '14': 1,
    '15': 1,
    '21': 2,
    '22': 2,
    '23': 2,
    '24': 2,
    '25': 2,
    '31': 3,
    '32': 3,
    '33': 3,
    '34': 3,
    '35': 3,
    '41': 4,
    '42': 4,
    '43': 4,
    '44': 4,
    '45': 4,
    '51': 5,
    '52': 5,
    '53': 5,
    '54': 5,
    '55': 5,
    'XX': None
    
}

tensKey = [None if pd.isna(p) else tensKey[p] for p in list(min_null_split.CAMEO_INTL_2015)]
min_null_split['TENS'] = tensKey

onesKey = {
    1: None,
    '11': 1,
    '12': 2,
    '13': 3,
    '14': 4,
    '15': 5,
    '21': 1,
    '22': 2,
    '23': 3,
    '24': 4,
    '25': 5,
    '31': 1,
    '32': 2,
    '33': 3,
    '34': 4,
    '35': 5,
    '41': 1,
    '42': 2,
    '43': 3,
    '44': 4,
    '45': 5,
    '51': 1,
    '52': 2,
    '53': 3,
    '54': 4,
    '55': 5,
    'XX': None
    
}

onesKey = [None if pd.isna(p) else onesKey[p] for p in list(min_null_split.CAMEO_INTL_2015)]
min_null_split['ONES'] = onesKey


In [ ]:
min_null_split.sample(10)
#min_null_split.shape

In [ ]:
#without encoding the data, i get an error when applying the PCA data.

from sklearn.preprocessing import OneHotEncoder
dummy_new = pd.get_dummies(min_null_split)
movement_change = dummy_new
new_encoded = min_null_split.replace({'M':0, 'A':1})


In [ ]:
new_encoded.head()

In [ ]:
new_encoded.reset_index(drop=True, inplace=True)
list(new_encoded)
new_encoded.shape

#### Discussion 1.2.2: Engineer Mixed-Type Features

In this section I was able to re-engineer two of the mixed catagories. I broke down the dimensionality of each varible and seperated them out. PRAEGENDE_JUGENDJAHRE, was seperated by decade, and movement. Then CAMEO_INTL_2015 was broken down into tens and ones column. After all that work i will delete these columns.

#### Step 1.2.3: Complete Feature Selection

In order to finish this step up, you need to make sure that your data frame now only has the columns that you want to keep. To summarize, the dataframe should consist of the following:
- All numeric, interval, and ordinal type columns from the original dataset.
- Binary categorical features (all numerically-encoded).
- Engineered features from other multi-level categorical features and mixed features.

Make sure that for any new columns that you have engineered, that you've excluded the original columns from the final dataset. Otherwise, their values will interfere with the analysis later on the project. For example, you should not keep "PRAEGENDE_JUGENDJAHRE", since its values won't be useful for the algorithm: only the values derived from it in the engineered features you created should be retained. As a reminder, your data should only be from **the subset with few or no missing values**.

In [ ]:
# Do whatever you need to in order to ensure that the dataframe only contains
# the columns that should be passed to the algorithm functions.

new_encoded.drop(['PRAEGENDE_JUGENDJAHRE', 'CAMEO_INTL_2015'], axis=1, inplace=True)


### Step 1.3: Create a Cleaning Function

Even though you've finished cleaning up the general population demographics data, it's important to look ahead to the future and realize that you'll need to perform the same cleaning steps on the customer demographics data. In this substep, complete the function below to execute the main feature selection, encoding, and re-engineering steps you performed above. Then, when it comes to looking at the customer data in Step 3, you can just run this function on that DataFrame to get the trimmed dataset in a single step.

In [ ]:
def clean_data():
    """
    Perform feature trimming, re-encoding, and engineering for demographics
    data
    
    INPUT: Demographics DataFrame
    OUTPUT: Trimmed and cleaned demographics DataFrame
    """
    from sklearn.preprocessing import OneHotEncoder
    
    customer = pd.read_csv('Udacity_CUSTOMERS_Subset.csv', sep= ';')
    feat_info = pd.read_csv('AZDIAS_Feature_Summary.csv', sep =';')
    #getting rid of nulls
    for x in range(len(feat_info)):
        null_data = feat_info.iloc[x]['missing_or_unknown']
        null_data = null_data.strip('[')
        null_data = null_data.strip(']')
        null_data = null_data.split(sep=',')
        null_data = [int(value) if (value!='X' and value!='XX' and value!='') else value for value in null_data]
        if null_data != ['']:
            customer = customer.replace({feat_info.iloc[x]['attribute']: null_data}, np.nan)

    #dropping columns   
    new_customer = customer.drop(labels = ['AGER_TYP', 'GEBURTSJAHR', 'TITEL_KZ', 'KK_KUNDENTYP', 'KBA05_BAUMAX', 'ALTER_HH'], axis =1 )


    #splitting data into subset, using smallest subset
    customer_min_null_split =  new_azdias[find_row_null<30].reset_index()
    customer_max_null_split =  new_azdias[find_row_null>=30].reset_index()
    #test2 = set(customer_min_null_split.PRAEGENDE_JUGENDJAHRE)  tween(20,30)].r


    #encoding
    col_to_drop =feat_info[feat_info['type'] == 'categorical']['attribute']
    for x in col_to_drop:
        if x in customer_min_null_split:
            customer_min_null_split.drop(x, inplace = True, axis = 1)

    
    
    #pulling out variables 
    decadesKey = {
    1: 40,
    2: 40,
    3: 50,
    4: 50,
    5: 60,
    6: 60,
    7: 60,
    8: 70,
    9: 70,
    10: 80,
    11: 80,
    12: 80,
    13: 80,
    14: 90,
    15: 90
    }

    decades = [None if pd.isna(p) else decadesKey[p] for p in list(customer_min_null_split.PRAEGENDE_JUGENDJAHRE)]
    customer_min_null_split['DECADE'] = decades

    movementsKey = {
    1: 'M',
    2: 'A',
    3: 'M',
    4: 'A',
    5: 'M',
    6: 'A',
    7: 'A',
    8: 'M',
    9: 'A',
    10: 'M',
    11: 'A',
    12: 'M',
    13: 'A',
    14: 'M',
    15: 'A'
    }

    movements = [None if pd.isna(p) else movementsKey[p] for p in list(customer_min_null_split.PRAEGENDE_JUGENDJAHRE)]
    customer_min_null_split['MOVEMENT'] = movements

    tensKey = {
    1: None,
    '11': 1,
    '12': 1,
    '13': 1,
    '14': 1,
    '15': 1,
    '21': 2,
    '22': 2,
    '23': 2,
    '24': 2,
    '25': 2,
    '31': 3,
    '32': 3,
    '33': 3,
    '34': 3,
    '35': 3,
    '41': 4,
    '42': 4,
    '43': 4,
    '44': 4,
    '45': 4,
    '51': 5,
    '52': 5,
    '53': 5,
    '54': 5,
    '55': 5,
    'XX': None
    }

    tensKey = [None if pd.isna(p) else tensKey[p] for p in list(customer_min_null_split.CAMEO_INTL_2015)]
    customer_min_null_split['TENS'] = tensKey

    onesKey = {
    1: None,
    '11': 1,
    '12': 2,
    '13': 3,
    '14': 4,
    '15': 5,
    '21': 1,
    '22': 2,
    '23': 3,
    '24': 4,
    '25': 5,
    '31': 1,
    '32': 2,
    '33': 3,
    '34': 4,
    '35': 5,
    '41': 1,
    '42': 2,
    '43': 3,
    '44': 4,
    '45': 5,
    '51': 1,
    '52': 2,
    '53': 3,
    '54': 4,
    '55': 5,
    'XX': None

    }

    onesKey = [None if pd.isna(p) else onesKey[p] for p in list(customer_min_null_split.CAMEO_INTL_2015)]
    customer_min_null_split['ONES'] = onesKey
    
    
    dummy_new = pd.get_dummies(customer_min_null_split)
    movement_change = dummy_new
    customer_encoded = customer_min_null_split.replace({'M':0, 'A':1})
    
   
    

    ##dropping columns
    customer_min_null_split.drop(['PRAEGENDE_JUGENDJAHRE', 'CAMEO_INTL_2015'], axis=1, inplace=True)
    return customer_encoded



## Step 2: Feature Transformation

### Step 2.1: Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features. Starting from this part of the project, you'll want to keep an eye on the [API reference page for sklearn](http://scikit-learn.org/stable/modules/classes.html) to help you navigate to all of the classes and functions that you'll need. In this substep, you'll need to check the following:

- sklearn requires that data not have missing values in order for its estimators to work properly. So, before applying the scaler to your data, make sure that you've cleaned the DataFrame of the remaining missing values. This can be as simple as just removing all data points with missing data, or applying an [Imputer](https://scikit-learn.org/0.16/modules/generated/sklearn.preprocessing.Imputer.html) to replace all missing values. You might also try a more complicated procedure where you temporarily remove missing values in order to compute the scaling parameters before re-introducing those missing values and applying imputation. Think about how much missing data you have and what possible effects each approach might have on your analysis, and justify your decision in the discussion section below.
- For the actual scaling function, a [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) instance is suggested, scaling each feature to mean 0 and standard deviation 1.
- For these classes, you can make use of the `.fit_transform()` method to both fit a procedure to the data as well as apply the transformation to the data at the same time. Don't forget to keep the fit sklearn objects handy, since you'll be applying them to the customer demographics data towards the end of the project.

In [ ]:
# If you've not yet cleaned the dataset of all NaN values, then investigate and
# do that now.

new_encoded.dropna(inplace=True)
new_encoded.shape

In [ ]:
#ensuring the data is ready to go 
new_encoded.isnull().sum()

In [ ]:
new_encoded.shape

In [ ]:
# Apply feature scaling to the general population demographics data.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
new_encoded = pd.DataFrame(scaler.fit_transform(new_encoded), columns = new_encoded.columns)

In [ ]:
new_encoded.head()

In [ ]:
new_encoded.reset_index(drop=True, inplace=True)

### Discussion 2.1: Apply Feature Scaling

In the above section I used StandardScaler to scale and transform the new_encoded azdias data set 

### Step 2.2: Perform Dimensionality Reduction

On your scaled data, you are now ready to apply dimensionality reduction techniques.

- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [ ]:
new_encoded.shape

In [ ]:
# Apply PCA to the data.
from sklearn.decomposition import PCA
my_pca = PCA()
#scaled_pca = my_pca.fit_transform(scaled_data)
scaled_pca =  my_pca.fit_transform(new_encoded)


In [ ]:
# Investigate the variance accounted for by each principal component.

my_pca.explained_variance_ratio_
components=np.arange(len(my_pca.explained_variance_ratio_))
v=my_pca.explained_variance_ratio_
plt.bar(components,v)

In [ ]:
# Re-apply PCA to the data while selecting for number of components to retain.

my_pca = PCA(n_components = 16)
scaled_pca = my_pca.fit_transform(new_encoded)

print(my_pca.components_.shape)
print(scaled_pca.shape)

In [ ]:
print(my_pca.components_)

### Discussion 2.2: Perform Dimensionality Reduction

In this section I ran the PCA and .fit_transform functions on the azdias encoded data. I decided to use 16 n_components for the data set. as it is 0.25 % of the encoded_data.

### Step 2.3: Interpret Principal Components

Now that we have our transformed principal components, it's a nice idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

As a reminder, each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

- To investigate the features, you should map each weight to their corresponding feature name, then sort the features according to weight. The most interesting features for each principal component, then, will be those at the beginning and end of the sorted list. Use the data dictionary document to help you understand these most prominent features, their relationships, and what a positive or negative value on the principal component might indicate.
- You should investigate and interpret feature associations from the first three principal components in this substep. To help facilitate this, you should write a function that you can call at any time to print the sorted list of feature weights, for the *i*-th principal component. This might come in handy in the next step of the project, when you interpret the tendencies of the discovered clusters.

In [ ]:
# Map weights for the first principal component to corresponding feature names
# and then print the linked values, sorted by weight.
# HINT: Try defining a function here or in a new cell that you can reuse in the
# other cells.

def finding_weights(my_pca, x):
    new_pandas_pca = pd.DataFrame(my_pca.components_, columns=list(new_encoded.columns))
    my_pca_weights = new_pandas_pca.iloc[x].sort_values(ascending=False)
    return my_pca_weights

In [ ]:
new_pandas_pca = pd.DataFrame(my_pca.components_, columns=list(new_encoded.columns))
new_pandas_pca.head()
new_pandas_pca.reset_index(drop=True, inplace=True)

In [ ]:
d0 = finding_weights(my_pca, 0)

In [ ]:
d0[0:5]

In [ ]:
weights = my_pca.components_ #verifying function works correctly on dataset
weights[0].max()

In [ ]:
# Map weights for the second principal component to corresponding feature names
# and then print the linked values, sorted by weight.
d1 = finding_weights(my_pca, 1)

In [ ]:
d1[0:5]

In [ ]:
# Map weights for the third principal component to corresponding feature names
# and then print the linked values, sorted by weight.
d2 = finding_weights(my_pca, 2)
d2[0:5]

### Discussion 2.3: Interpret Principal Components

From the above observation we can conclude that there is a positive relationship with the top results. for example:</br>

PLZ8_ANTG3 (Number of 6-10 family houses)</br>
PLZ8_ANTG4 (Number of 10+ family houses) </br>  
PLZ8_BAUMAX (Most common building type)</br>
These outcomes are all in the same region (PLZ8 region)and they have a postive correlation with eachother.




## Step 3: Clustering

### Step 3.1: Apply Clustering to General Population

You've assessed and cleaned the demographics data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- Use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Then, compute the average difference from each point to its assigned cluster's center. **Hint**: The KMeans object's `.score()` method might be useful here, but note that in sklearn, scores tend to be defined so that larger is better. Try applying it to a small, toy dataset, or use an internet search to help your understanding.
- Perform the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Use this fact to select a final number of clusters in which to group the data. **Warning**: because of the large size of the dataset, it can take a long time for the algorithm to resolve. The more clusters to fit, the longer the algorithm will take. You should test for cluster counts through at least 10 clusters to get the full picture, but you shouldn't need to test for a number of clusters above about 30.
- Once you've selected a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation. Make sure that you also obtain the cluster assignments for the general demographics data, since you'll be using them in the final Step 3.3.

In [ ]:
# Over a number of different cluster counts...
from sklearn.cluster import KMeans
X = my_pca.components_
ss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init =10, random_state = 0)
    kmeans.fit(X.reshape(-1,1))
    ss.append(kmeans.inertia_)
plt.plot(range(1,11),ss)
plt.show()

In [ ]:

kmeans = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 300, n_init =10, random_state = 0)
pred_y = kmeans.fit_predict(X)
plt.scatter(kmeans.cluster_centers_[:,0],kmeans.cluster_centers_[:,1],s =300, c='red')
plt.show()


In [ ]:
new_pandas_pca.shape

In [ ]:
# Investigate the change in within-cluster distance across number of clusters.
# HINT: Use matplotlib's plot function to visualize this relationship.

import sklearn.cluster as cluster
plt.plot(range(len(my_pca.explained_variance_ratio_)),np.cumsum(my_pca.explained_variance_ratio_), '-')

In [ ]:
# Re-fit the k-means model with the selected number of clusters and obtain
# cluster predictions for the general population demographics data.

azdias_model = KMeans(n_clusters=3).fit(new_pandas_pca)
azdias_pred = azdias_model.predict(new_pandas_pca)

### Discussion 3.1: Apply Clustering to General Population

Above I used Kmeans() to cluster the azdiaz encoded data set, i decided to use 8 clusters for the data.

### Step 3.2: Apply All Steps to the Customer Data

Now that you have clusters and cluster centers for the general population, it's time to see how the customer data maps on to those clusters. Take care to not confuse this for re-fitting all of the models to the customer data. Instead, you're going to use the fits from the general population to clean, transform, and cluster the customer data. In the last step of the project, you will interpret how the general population fits apply to the customer data.

- Don't forget when loading in the customers data, that it is semicolon (`;`) delimited.
- Apply the same feature wrangling, selection, and engineering steps to the customer demographics using the `clean_data()` function you created earlier. (You can assume that the customer demographics data has similar meaning behind missing data patterns as the general demographics data.)
- Use the sklearn objects from the general demographics data, and apply their transformations to the customers data. That is, you should not be using a `.fit()` or `.fit_transform()` method to re-fit the old objects, nor should you be creating new sklearn objects! Carry the data through the feature scaling, PCA, and clustering steps, obtaining cluster assignments for all of the data in the customer demographics data.

In [ ]:
# Load in the customer demographics data.
customer.head()
customer.describe()

In [ ]:
# Apply preprocessing, feature transformation, and clustering from the general
# demographics onto the customer data, obtaining cluster predictions for the
# customer demographics data.

cleaned_customer_data = clean_data()


In [ ]:
cleaned_customer_data.dropna(inplace=True)

In [ ]:
#Verifiying Clean_data() worked and datasets are even
print('AZDIAS data set',new_encoded.shape)
print('Customer data set',cleaned_customer_data.shape)

In [ ]:
customers_standardized = scaler.transform(customer_encoded)
pca_customers = my_pca.transform(customers_standardized)
customer_model = KMeans(n_clusters = 3)
customer_pred = customer_model.fit_predict(pca_customers)

### Step 3.3: Compare Customer Data to Demographics Data

At this point, you have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. In this final substep, you will compare the two cluster distributions to see where the strongest customer base for the company is.

Consider the proportion of persons in each cluster for the general population, and the proportions for the customers. If we think the company's customer base to be universal, then the cluster assignment proportions should be fairly similar between the two. If there are only particular segments of the population that are interested in the company's products, then we should see a mismatch from one to the other. If there is a higher proportion of persons in a cluster for the customer data compared to the general population (e.g. 5% of persons are assigned to a cluster for the general population, but 15% of the customer data is closest to that cluster's centroid) then that suggests the people in that cluster to be a target audience for the company. On the other hand, the proportion of the data in a cluster being larger in the general population than the customer data (e.g. only 2% of customers closest to a population centroid that captures 6% of the data) suggests that group of persons to be outside of the target demographics.

Take a look at the following points in this step:

- Compute the proportion of data points in each cluster for the general population and the customer data. Visualizations will be useful here: both for the individual dataset proportions, but also to visualize the ratios in cluster representation between groups. Seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) or [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function could be handy.
  - Recall the analysis you performed in step 1.1.3 of the project, where you separated out certain data points from the dataset if they had more than a specified threshold of missing values. If you found that this group was qualitatively different from the main bulk of the data, you should treat this as an additional data cluster in this analysis. Make sure that you account for the number of data points in this subset, for both the general population and customer datasets, when making your computations!
- Which cluster or clusters are overrepresented in the customer dataset compared to the general population? Select at least one such cluster and infer what kind of people might be represented by that cluster. Use the principal component interpretations from step 2.3 or look at additional components to help you make this inference. Alternatively, you can use the `.inverse_transform()` method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.
- Perform a similar investigation for the underrepresented clusters. Which cluster or clusters are underrepresented in the customer dataset compared to the general population, and what kinds of people are typified by these clusters?

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.

figure, (axa,axc) =(plt.subplots(1,2)) 
axa = sns.countplot(azdias_pred, ax= axa) #first graph is azdias 
axc = sns.countplot(customer_pred, ax=axc)#second graph is customer

In [ ]:
# What kinds of people are part of a cluster that is overrepresented in the
# customer data compared to the general population?

pca = PCA(n_components=8, whiten=True).fit(new_pandas_pca)
X_pca = pca.transform(new_pandas_pca)
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=8).fit(X_pca)
centers = pca.inverse_transform(kmeans.cluster_centers_)
#print(centers[2])


for label in range(kmeans.n_clusters):
    print(label,new_pandas_pca[kmeans.labels_ == label].mean(0))
    #print(label[index[1]])
    
#over represented is column#3(index 3 below) as in the customer data set this is very high, 
#but in the azdias data set it is a lot lower.
     

In [ ]:
# What kinds of people are part of a cluster that is underrepresented in the
# customer data compared to the general population?

#the under represented group is #1(index 1 in the above printout) as the bar is larger on the azdias graph, and smaller for the customer data set.

### Discussion 3.3: Compare Customer Data to Demographics Data

(Double-click this cell and replace this text with your own text, reporting findings and conclusions from the clustering analysis. Can we describe segments of the population that are relatively popular with the mail-order company, or relatively unpopular with the company?)

Based on the data above we can see that columns 3,4,and 6 are overrepresented from the coustomer vs general data. we can see that the outter columns 0 and 7 are equally represented by the mailers, and column one is underrepresented. Its rather unclear looking into the MD file what type of person these column represent. 

> Congratulations on making it this far in the project! Before you finish, make sure to check through the entire notebook from top to bottom to make sure that your analysis follows a logical flow and all of your findings are documented in **Discussion** cells. Once you've checked over all of your work, you should export the notebook as an HTML document to submit for evaluation. You can do this from the menu, navigating to **File -> Download as -> HTML (.html)**. You will submit both that document and this notebook for your project submission.